In [22]:
import tkinter as tk
from tkinter import filedialog, messagebox
import mne
import numpy as np
import os
"""
  * @author: dengyufeng
  * @Created on 2024/8/13 16:42
 """
class EEGProcessorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("EEG Processor")
        self.root.geometry("1000x800")  # 将窗口大小调整为1000x800

        # 创建并放置按钮
        self.load_button = tk.Button(root, text="加载EEG文件", command=self.load_eeg_file)
        self.load_button.pack(pady=10)

        self.batch_process_button = tk.Button(root, text="选择要处理的文件", command=self.select_files, state=tk.DISABLED)
        self.batch_process_button.pack(pady=10)

        self.process_button = tk.Button(root, text="处理数据", command=self.process_data, state=tk.DISABLED)
        self.process_button.pack(pady=10)

        self.result_label = tk.Label(root, text="", wraplength=950)  # 调整wraplength以适应放大的窗口
        self.result_label.pack(pady=10)

        self.save_button = tk.Button(root, text="保存文件", command=self.save_file, state=tk.DISABLED)
        self.save_button.pack(pady=10)

        self.close_button = tk.Button(root, text="关闭", command=self.close_app)
        self.close_button.pack(pady=10)

        # 创建一个框架用于组织tmin、tmax、基线、标注设置
        settings_frame = tk.Frame(root)
        settings_frame.pack(pady=10)

        # tmin 和 tmax 输入框
        tk.Label(settings_frame, text="tmin:").grid(row=0, column=0, padx=5, pady=5, sticky='e')
        self.tmin_entry = tk.Entry(settings_frame)
        self.tmin_entry.grid(row=0, column=1, padx=5, pady=5)

        tk.Label(settings_frame, text="tmax:").grid(row=0, column=2, padx=5, pady=5, sticky='e')
        self.tmax_entry = tk.Entry(settings_frame)
        self.tmax_entry.grid(row=0, column=3, padx=5, pady=5)

        self.set_tmin_tmax_button = tk.Button(settings_frame, text="设置 tmin 和 tmax", command=self.set_tmin_tmax)
        self.set_tmin_tmax_button.grid(row=0, column=4, padx=5, pady=5)

        # 基线设置输入框
        tk.Label(settings_frame, text="基线 (格式: '开始时间 结束时间' 或 'None'):").grid(row=1, column=0, padx=5, pady=5, sticky='e')
        self.baseline_entry = tk.Entry(settings_frame)
        self.baseline_entry.grid(row=1, column=1, padx=5, pady=5, columnspan=3)

        self.set_baseline_button = tk.Button(settings_frame, text="设置基线", command=self.set_baseline)
        self.set_baseline_button.grid(row=1, column=4, padx=5, pady=5)

        # 自定义标注输入框
        tk.Label(settings_frame, text="自定义标注 (格式: '标注名=事件编号', 例如 'T1=1 T2=2'):").grid(row=2, column=0, padx=5, pady=5, sticky='e')
        self.annotation_entry = tk.Entry(settings_frame)
        self.annotation_entry.grid(row=2, column=1, padx=5, pady=5, columnspan=3)

        self.set_annotation_button = tk.Button(settings_frame, text="设置自定义标注", command=self.set_annotations)
        self.set_annotation_button.grid(row=2, column=4, padx=5, pady=5)

        # 保存文件名输入框
        self.filename_entries = {}

        # 默认 tmin, tmax, baseline 值
        self.tmin = None
        self.tmax = None
        self.baseline = None

        self.file_paths = []  # 存储选择的文件路径
        self.annotations = {}
        self.epochs_data = {}

        # 文件格式选择
        self.file_format_label = tk.Label(root, text="选择EEG文件格式:")
        self.file_format_label.pack(pady=5)
        self.file_format_var = tk.StringVar(value="edf")
        self.edf_radio = tk.Radiobutton(root, text="EDF", variable=self.file_format_var, value="edf")
        self.edf_radio.pack(pady=5)
        self.bdf_radio = tk.Radiobutton(root, text="BDF", variable=self.file_format_var, value="bdf")
        self.bdf_radio.pack(pady=5)
        self.fif_radio = tk.Radiobutton(root, text="FIF", variable=self.file_format_var, value="fif")
        self.fif_radio.pack(pady=5)

    def load_eeg_file(self):
        filetypes = [("EEG files", "*.edf *.bdf *.fif")]
        file_path = filedialog.askopenfilename(filetypes=filetypes)
        if file_path:
            self.file_paths = [file_path]
            messagebox.showinfo("文件加载", "文件已加载成功")
            self.process_button.config(state=tk.NORMAL)
            self.batch_process_button.config(state=tk.NORMAL)

    def select_files(self):
        filetypes = [("EEG files", "*.edf *.bdf *.fif")]
        file_paths = filedialog.askopenfilenames(filetypes=filetypes)
        if file_paths:
            self.file_paths = list(file_paths)
            messagebox.showinfo("文件选择", f"已选择 {len(self.file_paths)} 个文件")
            self.process_button.config(state=tk.NORMAL)
            self.save_button.config(state=tk.NORMAL)

    def process_data(self):
        if self.tmin is None or self.tmax is None:
            messagebox.showwarning("错误", "请先设置 tmin 和 tmax")
            return
        
        if not self.file_paths:
            messagebox.showwarning("错误", "请先选择要处理的EEG文件")
            return
        
        if not self.annotations:
            messagebox.showwarning("错误", "请先设置自定义标注")
            return
        
        self.epochs_data = {}
        for file_path in self.file_paths:
            try:
                self.process_single_file(file_path, self.file_format_var.get())
            except Exception as e:
                messagebox.showerror("处理错误", f"处理文件 {os.path.basename(file_path)} 时出错: {e}")
                continue
        
        # 计算每个标注的数据形状
        result_text = ""
        for name, data_list in self.epochs_data.items():
            if data_list:
                # 假设每个标注的所有数据维度一致，取第一个数据项来获取形状
                sample_shape = data_list[0].shape
                result_text += f"{name} Epochs形状: {sample_shape}\n"
        
        self.result_label.config(text=result_text)

        messagebox.showinfo("处理完成", "数据已处理。")
        self.save_button.config(state=tk.NORMAL)

    def process_single_file(self, file_path, file_format):
        # 读取EEG文件
        if file_format == "edf":
            raw = mne.io.read_raw_edf(file_path, preload=True)
        elif file_format == "bdf":
            raw = mne.io.read_raw_bdf(file_path, preload=True)
        elif file_format == "fif":
            raw = mne.io.read_raw_fif(file_path, preload=True)
        else:
            raise ValueError("不支持的文件格式")
        
        # 从标注中提取事件
        events, event_id = mne.events_from_annotations(raw, event_id=self.annotations)
        
        # 选择EEG通道
        picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
        
        # 提取和处理事件数据
        for name, event_code in self.annotations.items():
            epochs = mne.Epochs(raw, events, event_id={name: event_code}, tmin=self.tmin, tmax=self.tmax, baseline=self.baseline, picks=picks, preload=True)
            if name not in self.epochs_data:
                self.epochs_data[name] = []
            self.epochs_data[name].append((epochs.get_data() * 1e6).astype(np.float32))  # 将数据转换为微伏

    def set_tmin_tmax(self):
        try:
            self.tmin = float(self.tmin_entry.get())
            self.tmax = float(self.tmax_entry.get())
            messagebox.showinfo("设置成功", f"tmin 设置为 {self.tmin}\ntmax 设置为 {self.tmax}")
        except ValueError:
            messagebox.showerror("输入错误", "请确保 tmin 和 tmax 输入的是有效的数字")

    def set_baseline(self):
        baseline_text = self.baseline_entry.get().strip()
        if baseline_text.lower() == "none":
            self.baseline = None
        else:
            try:
                self.baseline = tuple(float(x) for x in baseline_text.split())
            except ValueError:
                messagebox.showerror("输入错误", "请确保基线输入的是有效的时间范围或 'None'")
                return
        messagebox.showinfo("设置成功", f"基线设置为: {self.baseline}")

    def set_annotations(self):
        annotation_text = self.annotation_entry.get()
        try:
            self.annotations = {}
            for item in annotation_text.split():
                name, id_ = item.split('=')
                self.annotations[name.strip()] = int(id_.strip())
            messagebox.showinfo("设置成功", f"自定义标注已设置为: {self.annotations}")

            # 为每个标注生成文件名输入框
            for widget in self.filename_entries.values():
                widget.pack_forget()  # 清除之前的输入框
            
            self.filename_entries.clear()

            for name in self.annotations.keys():
                label = tk.Label(self.root, text=f"输入{name}的保存文件名:")
                label.pack(pady=5)
                entry = tk.Entry(self.root)
                entry.pack(pady=5)
                self.filename_entries[name] = entry
        except ValueError:
            messagebox.showerror("输入错误", "请确保标注格式正确，如 'T1=1 T2=2'")

    def save_file(self):
        if self.epochs_data:
            save_dir = filedialog.askdirectory()
            if save_dir:
                for name, data_list in self.epochs_data.items():
                    filename = self.filename_entries[name].get().strip()
                    if not filename:
                        messagebox.showerror("保存错误", f"请为{name}提供文件名")
                        return
                    for i, data in enumerate(data_list):
                        save_path = f"{save_dir}/{filename}_part{i+1}.npz"
                        np.savez(save_path, data=data)
                messagebox.showinfo("保存成功", f"文件已保存到: {save_dir}")
        else:
            messagebox.showwarning("保存错误", "没有可保存的数据")

    def close_app(self):
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = EEGProcessorApp(root)
    root.mainloop()


Extracting EDF parameters from C:\Users\pc\Desktop\李佳\S002\S002R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Used Annotations descriptions: [np.str_('T1'), np.str_('T2')]
Not setting metadata
8 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 8 events and 961 original time points ...
1 bad epochs dropped
Not setting metadata
7 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 7 events and 961 original time points ...
0 bad epochs dropped


C:\Users\pc\AppData\Local\Temp\ipykernel_16340\3755149150.py:161: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  self.epochs_data[name].append((epochs.get_data() * 1e6).astype(np.float32))  # 将数据转换为微伏
C:\Users\pc\AppData\Local\Temp\ipykernel_16340\3755149150.py:161: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  self.epochs_data[name].append((epochs.get_data() * 1e6).astype(np.float32))  # 将数据转换为微伏


Extracting EDF parameters from C:\Users\pc\Desktop\李佳\S002\S002R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Used Annotations descriptions: [np.str_('T1'), np.str_('T2')]
Not setting metadata
8 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 8 events and 961 original time points ...
1 bad epochs dropped
Not setting metadata
7 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 7 events and 961 original time points ...
0 bad epochs dropped
Extracting EDF parameters from C:\Users\pc\Desktop\李佳\S002\S002R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Used Annotations descriptions: [np.str_('T1'), np.str_('T2')]
Not setting metadata
8 m

C:\Users\pc\AppData\Local\Temp\ipykernel_16340\3755149150.py:161: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  self.epochs_data[name].append((epochs.get_data() * 1e6).astype(np.float32))  # 将数据转换为微伏
C:\Users\pc\AppData\Local\Temp\ipykernel_16340\3755149150.py:161: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  self.epochs_data[name].append((epochs.get_data() * 1e6).astype(np.float32))  # 将数据转换为微伏
C:\Users\pc\AppData\Local\Temp\ipykernel_16340\3755149150.py:161: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  self.epochs_data[name].append((epochs.get_data() * 1e6).astype(np.float32))  # 将数据转换为微伏
C:\Users\pc\AppData\Local\Temp\ipykernel_16340\3755149150.py:161: FutureWarning: The current default of copy=False will c

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Used Annotations descriptions: [np.str_('T1'), np.str_('T2')]
Not setting metadata
8 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 8 events and 961 original time points ...
1 bad epochs dropped
Not setting metadata
7 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 7 events and 961 original time points ...
0 bad epochs dropped
Extracting EDF parameters from C:\Users\pc\Desktop\李佳\S002\S002R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Used Annotations descriptions: [np.str_('T1'), np.str_('T2')]
Not setting metadata
7 matching events found
Applying baseline correction (mode: mean)
0 projectio

C:\Users\pc\AppData\Local\Temp\ipykernel_16340\3755149150.py:161: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  self.epochs_data[name].append((epochs.get_data() * 1e6).astype(np.float32))  # 将数据转换为微伏
C:\Users\pc\AppData\Local\Temp\ipykernel_16340\3755149150.py:161: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  self.epochs_data[name].append((epochs.get_data() * 1e6).astype(np.float32))  # 将数据转换为微伏
C:\Users\pc\AppData\Local\Temp\ipykernel_16340\3755149150.py:161: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  self.epochs_data[name].append((epochs.get_data() * 1e6).astype(np.float32))  # 将数据转换为微伏
C:\Users\pc\AppData\Local\Temp\ipykernel_16340\3755149150.py:161: FutureWarning: The current default of copy=False will c